## Table Of Contents
* [Purpose](#Purpose)
* [Dependencies](#Dependencies)
    * [Python and Anaconda installation](#Python-and-Anaconda-installation)
    * [A Python kernel for Jupyter notebooks](#A-Python-kernel-for-Jupyter-notebooks)
    * [Load the Python Packages](#Load-the-Python-Packages)
* [Inputs](#inputs)
* [Outputs](#Outputs)
    * [Tabular Data](#Tabular-Data)
    * [Map](#Map)
    * [Bar Chart](#Bar-Chart)
* [Code](#Code)
* [Provenance](#Provenance)
* [Citations](#Citations)






    

# Purpose
The purpose of this analysis package is to provide greater understanding of plant-pollinator networks and land management activities that improve pollinator habitat through Data-Visualization using Python Packages. The aim of the Pollinator Library is to support management and research of plant-pollinator systems by documenting, synthesizing, and disseminating information on flowers that are utilized by pollinators and other insects.

# Dependencies

Several software packages are required to run this scientific code in a Jupyter notebook.

## Python and Anaconda installation <a class="anchor" id="Python-and-Anaconda-installation"></a>

Instructions on installing the software environment is available at the(https://www.anaconda.com/download/). Conda is a powerful package manager and environment manager that you use with command line commands at the Anaconda Prompt for Windows, or in a Terminal window for macOS or Linux..
   
## A Python kernel for Jupyter notebooks <a class="anchor" id="A-Python-kernel-for-Jupyter-notebooks"></a>

 This notebook uses [Python3 Kernel](https://ipython.readthedocs.io/en/latest/install/kernel_install.html).  

## Load the Python Packages <a class="anchor" id="Load-the-Python-Packages"></a>

In [1]:
import io, os, sys, types

In [2]:
import nbformat

from IPython import get_ipython
from IPython.core.interactiveshell import InteractiveShell
from collections import defaultdict,Counter, namedtuple
from operator import itemgetter, attrgetter
from urllib.request import Request, urlopen
from pandas.io.json import json_normalize
import json
import pandas as pd
import shlex
from IPython.display import display, Javascript, HTML
import folium
import plotly.plotly as py
import plotly.graph_objs as go
import itertools 
from itertools import groupby
import iplotter
from ipysankeywidget import SankeyWidget
import plotly
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='abhimike17', api_key='YtVTWLxeARvUWeNYurX7')
from nbpackage import mynotebook


In [3]:
# Import the REST API Web Service and filter the data
api = Request('http://127.0.0.1:8000/pdatas/')
Iapi = Request('http://127.0.0.1:8000/locations/')
Papi = Request('http://127.0.0.1:8000/locationspsearch/')


json_obj = urlopen(api)
pdata = json.load(json_obj)

Ijson_obj = urlopen(Iapi)
locI = json.load(Ijson_obj)

Pjson_obj = urlopen(Papi)
locP = json.load(Pjson_obj)


In [4]:
yi = filter(lambda x:x['insecttaxon'] == "Bombus rufocinctus", locI)
xi = list(yi)
lo = []
for k in xi:
    lo.append(k['latitude'])

# Inputs
The operational Pollinator Analysis Prototype retrieves data in real-time from the (https://www.npwrc.usgs.gov/pollinator/)'s REST API Web Processing Service.

In [5]:
# #Mention the Taxon for search and get the desired output
y = filter(lambda x:x['insecttaxon'] == "Bombus rufocinctus", pdata)
x = list(y)
df = pd.DataFrame(x)
anp = df.groupby(['insectgenus','planttaxon']).size().reset_index(name = 'count')
anp.columns = ['source','target','value']
xbyz = anp.T.to_dict().values()

xcf = []

for k in xbyz:
    if k not in xcf:
        xcf.append(k)

In [6]:
links = xcf

In [7]:
w = SankeyWidget(links=links, margins=dict(top=0, bottom=0, left=100, right=200))

In [16]:
w

SankeyWidget(links=[{'source': 'Bombus', 'target': 'Amorpha canescens', 'value': 2}, {'source': 'Bombus', 'tar…

In [9]:
# Get the latitude and longitude position of the taxon 

lat = Counter(k['latitude'] for k in xi)
long = Counter(k['longitude'] for k in xi)

latlist = []
longlist = []
count = []

for k in xi:
#     if k not in latlist:
        latlist.append(k['latitude'])

for k in xi:
#     if k not in longlist:
        longlist.append(k['longitude'])
# Get the interation list of Taxon depending on the search
p = Counter(k['planttaxon'] for k in xi)

# list of Plants
i = []
for k in xi:
        i.append(k['planttaxon'])


# Outputs

## Tabular Data <a class="anchor" id="Tabular-Data"></a>

In [10]:
#Data of Location and taxon interaction
df_counters = pd.DataFrame(
    {
     'latitude' : latlist,
     'longitude' : longlist,
        'planttaxon': i
    })


## Map
Produce a Leaflet Map of all modeled location results for Pollinator interactions.

In [11]:
locations = df_counters[['latitude','longitude']]
locationlist=locations.values.tolist()
len(locationlist)
newlist = []
for k in locationlist:
    j = [float(i) for i in k]
    newlist.append(j)
# len(locationlist)


In [12]:
# produce a Leaflet map with all Taxon locations
map = folium.Map(location=[39.048603, -94.116578], zoom_start=4, tiles='Stamen Terrain'
)
for point in range(0, len(locationlist)):
    folium.Marker(newlist[point], popup=df_counters['planttaxon'][point]).add_to(map)
map

## Bar Chart <a class="anchor" id="Bar-Chart"></a> 
Produce bar chart of the modeled results.

In [13]:
# Interaction of each plant
count = []
for k in xi:
    count.append(k['count'])

    j = []
for k in xi:
    j.append(k['planttaxon'])
    
data = [go.Bar(
            x=count,
            y=j,
            orientation = 'h'
)]


layout = go.Layout(
    autosize=True,
    width=800,
    height=800,
    yaxis=go.layout.YAxis(
        title='',
      
        tickmode='array',
        automargin=True,
        titlefont=dict(size=10),
    ),
  
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='automargin')
# py.iplot(data, filename='horizontal-bar')



In [14]:
def landuse(name):
#     name = "USFWS National Wildlife Refuge"
    zy = filter(lambda x:x['landuse'] == name, xi)
    zyp = list(zy)
    
    pl = []
    for k in zyp:
            pl.append(k['planttaxon'])

    jl = []
    for k in zyp:
        jl.append(k['count'])
        
   
    return pl, jl
mn, nm = landuse("USFWS National Wildlife Refuge")
man, nam = landuse("USDA Conservation Reserve Program")
abc, bac = landuse("Rangeland")

In [15]:
trace1 = go.Bar(
    y=man,
    x=nam,
    name='USDA Conservation Reserve Program',
    orientation = 'h',
    marker = dict(
        color = 'rgba(128, 0, 0, 0.60)',
        line = dict(
            color = 'rgba(128, 0, 0, 0.60)',
            width = 3)
    )
)
trace2 = go.Bar(
    y=mn,
    x=nm,
    name='USFWS National Wildlife Refuge',
    orientation = 'h',
    marker = dict(
        color = 'rgba(58, 71, 80, 0.6)',
        line = dict(
            color = 'rgba(58, 71, 80, 1.0)',
            width = 3)
    )
)
trace3 = go.Bar(
    y=abc,
    x=bac,
    name='Rangeland',
    orientation = 'h',
    marker = dict(
        color = 'rgba(0, 0, 128, 0.6)',
        line = dict(
            color = 'rgba(0, 0, 128, 1.0)',
            width = 3)
    )
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='stack',
    yaxis=go.layout.YAxis(
        title='',
      
        tickmode='array',
        automargin=True,
        titlefont=dict(size=20),
    ),
    
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='marker-h-bar')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~abhimike17/0 or inside your plot.ly account where it is named 'marker-h-bar'


# Code
Code is available at the 

# Provenance


# Citations
R Code for Otto, CRV, RB Bryant, NH Euliss Jr., S O'Dell, R Bush, and MD Smart. (2017). Using Publicly Available Data to Quantify Plant-Pollinator Interactions and Evaluate Conservation Seeding Mixes in the Northern Great Plains ###Analysis of data from Pollinator Library

R Code prepared by Clint Otto, US Geological Survey, Northern Prairie Wildlife Research Center

Analysis started on 06.20.2016

Code altered to provide Dan Wieferich with an example of how to take data from the Library,
and provide users with graphs and descriptive data. 02.02.2018

